In [10]:
#!pip install albumentations
#!pip install imgaug

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm 
import time
import os
from PIL import Image
import cv2 
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
#from albumentations.augmentations.transforms import HorizontalFlip,VerticalFlip

% matplotlib inline

In [52]:
FACIAL_LANDMARKS_68_IDXS = dict(
    [("mouth", (48, 68)),
    ("inner_mouth", (60, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))]
)

def flip_keypoints(kp, img_width):
    flipped_kp = kp.copy()
    
    for key, (low, high) in FACIAL_LANDMARKS_68_IDXS.items():
        for i in range((high-low)): 
            # flip order
            flipped_kp[low+i] = kp[high-1-i]
            # flip location
            flipped_kp[low+i, 0] = img_width-flipped_kp[low+i, 0]   # img_shape[1]

    return flipped_kp

In [59]:
def parse_keypoints(kp_raw):
    data = list(kp_raw.apply(eval))
    return np.array(data)
    
class TwinPairsDataset(Dataset):
    def __init__(self, dataroot, df_pairs, df_views, transform, keypoints=False):
        '''
        dataroot: path to folder with items
        df_pairs: pd dataframe containing pairs of ids and a correspodind label:
                    'Same', 'Fraternal', 'Identical', 'UnknownTwinType',
                    'IdenticalMirror', 'Sibling', 'IdenticalTriplet'
        df_views: pd dataframe containing list of available for each id in the dataset                    
        transform: torchvision transform
        '''
        self.dataroot = dataroot
        self.df_pairs = df_pairs
        self.df_views = df_views
        self.transform = transform
        self.keypoints = keypoints
        #if self.keypoints:
        #    self.flip=VerticalFlip()#HorizontalFlip()
    def __getitem__(self, index):
        sample={}
        
        id_1, id_2  = self.df_pairs.iloc[index][['id_1', 'id_2']].values
        print(id_1, id_2)
        id_1 = str(id_1)
        id_2 = str(id_2)
        label = int(self.df_pairs.iloc[index].label=='Same')
        print('Same' if label==1 else 'Twin')
        if id_1==id_2:
             view_1, view_2 = np.random.choice(self.df_views.loc[id_1]['filename'], size=2, replace=False) 
        else:
            view_1 = np.random.choice(self.df_views.loc[id_1]['filename'])
            view_2 = np.random.choice(self.df_views.loc[id_2]['filename'])

        path_1 = os.path.join(self.dataroot, id_1, view_1)
        path_2 = os.path.join(self.dataroot, id_2, view_2)
        
        img_1 = Image.open(path_1)
        img_2 = Image.open(path_2)
        
        if self.keypoints:
            kp_df_1 = pd.read_csv(self.dataroot+f'{id_1}/keypoints.csv')
            kp_df_2 = pd.read_csv(self.dataroot+f'{id_2}/keypoints.csv')
            keypoints_1 = parse_keypoints(kp_df_1[view_1])
            keypoints_2 = parse_keypoints(kp_df_2[view_2])
            
        
        
        if label==1:
            img_1 = self.transform['twin'](img_1)
            img_2 = self.transform['same'](img_2)
            if self.keypoints:
                #print(keypoints_2.shape)
                #keypoints_2=self.flip.apply_to_keypoint(keypoints_2[:,0],keypoints_2[:,1])
                #keypoints_2=self.flip.apply_to_keypoint(keypoints_2,rows=keypoints_2.shape[0], cols=keypoints_2.shape[1])
                keypoints_2=flip_keypoints(keypoints_2,img_2.shape[1])
        else:
            img_1 = self.transform['twin'](img_1)
            img_2 = self.transform['twin'](img_2)
        
        
        
        if self.keypoints:    
            sample['keypoints_1'] = keypoints_1
            sample['keypoints_2'] = keypoints_2
            #print(keypoints_2.shape)
            #print(keypoints_2)
        sample['img_1']=img_1
        sample['img_2']=img_2
        sample['label']=label
        
        return sample
    
    def __len__(self):
        return self.df_pairs.shape[0]

In [60]:
data_transform_train ={
    'same': transforms.Compose([
            transforms.RandomHorizontalFlip(p=1),
            transforms.CenterCrop(512),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'twin': transforms.Compose([
            transforms.CenterCrop(512),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
                }

data_transform_val={
    'same': transforms.Compose([
            transforms.CenterCrop(512),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'twin': transforms.Compose([
            transforms.CenterCrop(512),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
                }

## Train

In [77]:
views = pd.read_csv('./twins_aligned_cropped_resized/df/views.csv', index_col=0)
views.index = views.index.astype(str)
views.filename = views.filename.apply(eval)
pairs = pd.read_csv('./pairs_train.csv')
direct='./twins_aligned_cropped_resized/'

In [78]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

#### Load pretrained model

In [79]:
dataset = TwinPairsDataset(direct, pairs, views,transform=data_transform_train, keypoints=True)
dataloader = DataLoader(dataset, batch_size=10,shuffle=True)

sample=dataset[0]


90308 90308
Same


In [80]:
from model import MobileFaceNet

model = MobileFaceNet(embedding_size=512)

PATH = os.path.join(os.getcwd(),"./model_mobilefacenet.pth")

checkpoint = torch.load(PATH,map_location=device)

model.load_state_dict(checkpoint)

In [81]:
class ModelBottom(nn.Module):
    def __init__(self, original_model):
        super(ModelBottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        
    def forward(self, x):
        output = self.features(x)
        return output
model_conv = ModelBottom(model)
outputs = model_conv(sample['img_1'])
hid_size=outputs.data.shape[1]
fc_layers = nn.Sequential(
                      nn.Linear(hid_size, 500),
                      nn.Dropout(0.5),
                      nn.ReLU(inplace=True), 
                      nn.Linear(500,5),
                      nn.Dropout(0.5),
                      nn.ReLU(inplace=True))

class myModel(nn.Module):
        def __init__(self):
            super(myModel,self).__init__()
            self.model_start = model_conv
            self.fc_layers = nn.Sequential(
                      nn.Linear(hid_size, 500),
                      nn.Dropout(0.5),
                      nn.ReLU(inplace=True), 
                      nn.Linear(500,5),
                      nn.Dropout(0.5),
                      nn.ReLU(inplace=True))
   	    
        def forward(self,x):
            out1 = self.model_start(x)
            return self.fc_layers(out1)
    

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 3, 3], but got 3-dimensional input of size [3, 697, 512] instead

In [74]:
PATH='./11_epoch.pt'
model=myModel()
model = model.load_state_dict(torch.load(PATH, map_location=device))#torch.load(PATH,map_location=device)

model.eval()

hid_size=141
classifier=nn.Sequential(
                      nn.Linear(hid_size, hid_size),
                      nn.Dropout(0.3),
                      nn.ReLU(), 
                      nn.Linear(hid_size, hid_size),
                      nn.Dropout(0.3),
                      nn.ReLU(),
                      nn.Linear(hid_size, 2),
                      nn.Dropout(0.3),
                      nn.ReLU()
                      )

NameError: name 'myModel' is not defined

In [65]:
def train(model,model_classifier, dataloader, optimizer,criterion, device, n_epochs=100):
    model.to(device)
    classifier.to(device)
    loss_hist=[]
    for epoch in range(n_epochs):
        optimzer.zero_grad()
        loss=0
        for i,sample in enumerate(dataloader):
            lendmark_1=sample['keypoints_1'].to(device).view(-1)
            lendmark_2=sample['keypoints_2'].to(device).view(-1)
            print(lendmark_2.shape)
            img_1=sample['img_1'].to(device)
            img_2=sample['img_2'].to(device)
            label=sample['label'].to(device)
            out1=model(img_1)
            out2=model(img_2)
            out=torch.cat(out1,lendmark_1)-torch.cat(out2,lendmark_2)
            final_out=model_classifier(out)
            
            loss+=criterion(final_out,lablel)  # need to make correct
        loss.backward()
        optimizer.step()
        loss_hist.append(loss.item())
        plt.plot(loss_hist)
        plt.title('loss')
        plt.show()
        

In [ ]:
optimizer = optim.Adam(classifier.parameters(),lr=0.01)
criterion=nn.MSELoss()

train(model,classifier,dataloader, optimizer,criterion,device,100)